# RVC AI Cover Maker (separate parts version)
Created by [ShiromiyaG](https://github.com/ShiromiyaG)
- Colab inspired on [AICoverGen](https://github.com/SociallyIneptWeeb/AICoverGen) by [SociallyIneptWeeb](https://github.com/SociallyIneptWeeb)

In [ ]:
import os
import subprocess
from google.colab import drive
from pathlib import Path
import requests
from IPython.display import clear_output
import subprocess
import fileinput
import zipfile
import shutil

drive.mount('/content/drive')
Path('/content/drive/MyDrive/RVC AI Cover Maker Input').mkdir(parents=True, exist_ok=True)
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
os.chdir("/content")
# @title # Install
# @markdown ##### Yes, it takes time
# @markdown ##### Leave the extra files enabled only if you are going to use ensemble on vocals
voc_extra_files = False #@param {type:"boolean"}
# @markdown ##### To find bugs, you have the option of not clearing the output **(Causes crashes)**
not_clean_outputs = False #@param {type:"boolean"}
!sudo apt install ffmpeg python3.10-venv
!pip install poetry pedalboard
!poetry config virtualenvs.create false
Path('/content/arquivos').mkdir(parents=True, exist_ok=True)

%cd python-audio-separator
!pip install "audio-separator[gpu]" pyrubberband yt-dlp[default] wget git+https://github.com/IAHispano/gdown
if not_clean_outputs != True:
  clear_output()
%cd ../

if voc_extra_files == True:
  !git clone https://github.com/ZFTurbo/Music-Source-Separation-Training.git
  !wget -P /Music-Source-Separation-Training/models "https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_segm_models_sdr_9.77.ckpt"
  !wget -P /Music-Source-Separation-Training/models "https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/config_vocals_segm_models.yaml"
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/spectograma.py" # Spectograma
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/karokee_4band_v2_sn.pth" # karokee
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/download_checks.json" # download_checks
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/reverbpedalboard.py" # reverb_pedalboard
!wget -P ./arquivos "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/utils.py" # utils
if not_clean_outputs != True:
  clear_output()
!git clone https://github.com/shirounanashi/OrpheusDL.git
%cd OrpheusDL
if not_clean_outputs != True:
  !poetry install --no-root -q > /dev/null 2>&1
  clear_output()
else:
  !poetry install --no-root
%cd ../
!git clone https://git.ovosimpatico.com/ovosimpatico/orpheusdl-deezer.git ./OrpheusDL/modules/deezer
!wget -P OrpheusDL/config "https://github.com/ShiromiyaG/RVC-AI-Cover-Maker/releases/download/Dependencies/settings.json"
if not_clean_outputs != True:
  clear_output()
!git clone https://github.com/shirounanashi/RVC_CLI.git
%cd RVC_CLI
if not_clean_outputs != True:
  !poetry install --no-root > /dev/null 2>&1
  clear_output()
else:
  !poetry install --no-root
%cd ../
!wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/fcpe.pt"
!wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/hubert_base.pt"
!wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/rmvpe.pt"
#!pip --disable-pip-version-check install beautifulsoup4 ffmpeg ffmpeg-python==0.1.18 numpy==1.23.5 requests==2.31.0 tqdm faiss-cpu==1.7.3 librosa==0.9.1 pydub==0.25.1 pyworld==0.3.4 praat-parselmouth==0.4.2 resampy==0.4.2 scipy==1.11.1 sounddevice==0.4.6 soundfile==0.12.1 torchaudio==2.1.1 praat-parselmouth noisereduce fairseq numba onnxruntime onnxruntime_gpu==1.15.1 torch==2.1.1 torchcrepe==0.0.21 torchgen>=0.0.1 torch_directml torchvision==0.16.1 einops local-attention matplotlib==3.7.2 tensorboard ffmpy==0.3.1 tensorboardX edge-tts==6.1.9 pydantic fastapi uvicorn
!pip uninstall onnxruntime-gpu -y
!python -m pip --disable-pip-version-check -q install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
if not_clean_outputs != True:
  clear_output()

# Send audio file

In [ ]:
#@title ### Local file upload
from google.colab import files
import os
import fnmatch
def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if file.endswith(".mp3") or file.endswith(".flac") or file.endswith(".wav"):
                caminho_arquivo = os.path.join(root, file)
                arquivos.append(caminho_arquivo)
    return arquivos
#uploaded = files.upload()
uploaded = procurar_arquivos("/content")
for arquivo in uploaded:
  !cp arquivo "/content/musicas/arquivos-originais"

In [ ]:
#@title ### File from Google Drive
drive_path = "/content/drive/MyDrive/RVC AI Cover Maker Input" #@param {type:"string"}
import os
import fnmatch

def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if fnmatch.fnmatch(file, '*.mp3') or fnmatch.fnmatch(file, '*.flac') or fnmatch.fnmatch(file, '*.wav'):
                arquivos.append(os.path.join(root, file))
    return arquivos

arquivos_encontrados = procurar_arquivos(drive_path)

for arquivo in arquivos_encontrados:
    !cp drive_path "/content/musicas/arquivos-originais"

In [ ]:
#@title ### Download file from YouTube or Deezer
# @markdown #### Select one of the options to download a song
link_of_yt = "" #@param {type:"string"}
link_of_deezer = "" #@param {type:"string"}
# @markdown ###### If you are going to use Deezer to download
bf_secret = "" #@param {type:"string"}
track_url_key = "" #@param {type:"string"}
arl = "" #@param {type:"string"}

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

if link_of_yt != "":
  !yt-dlp --extract-audio -o "/content/musicas/arquivos-originais/%(title)s.%(ext)s"  --audio-format mp3 --audio-quality 0 "{link_of_yt}"
  if not_clean_outputs != True:
    clear_output()
  arquivo = get_last_modified_file(input_folder)
  !cp "{arquivo}" "/content/musicas/arquivos-originais"

if link_of_deezer != "":
  if bf_secret != "" and track_url_key != "" and arl != "":
    %cd OrpheusDL
    !python orpheus.py "{link_of_deezer}"
    if not_clean_outputs != True:
      clear_output()
    %cd ../
else:
  raise ValueError("Você precisa fornecer a bf secret, track url key arl da sua conta Deezer Premium")

#Programs

In [ ]:
#@title ### Remove Backing Vocals e Echo/Reverb
from IPython.display import Audio, display, clear_output
input_folder = "/content/musicas/arquivos-originais"
no_back_folder = "/content/musicas/sem-back"
no_inst_folder = "/content/musicas/sem-intrumental"
output_folder = "/content/musicas/output-folder"
output_drive = "/content/drive/MyDrive/RVC AI Cover Maker"

def verificar_pasta_vazia(caminho_pasta):
    if not os.listdir(caminho_pasta):
        raise ValueError("You haven't sent an audio file!")

def get_last_modified_file(directory, filter=''):
  arquivos = glob(directory + "/*")
  if filter != '':
      arquivos = [arquivo for arquivo in arquivos if filter in arquivo]
  if arquivos:
      return max(arquivos, key=os.path.getmtime)
  else:
      return None

def convert_to_mp3(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)
    audio.export(output_file, format="mp3")

try:
    verificar_pasta_vazia(input_folder)
except ValueError as e:
    print(e)

filename = get_last_modified_file(input_folder)
no_inst_output = os.path.join(no_inst_folder, filename)
print(f"{filename} processing with MDX23C-8KFFT-InstVoc_HQ is over!")
subprocess.run([
    'audio-separator',
    f"{no_inst_output}",
    '--model_file_dir', "/content/arquivos",
    '-m', 'karokee_4band_v2_sn.pth',
    '--vr_window_size', '320',
    '--vr_aggression', '5',
    "--normalization", "1.0",
    "--vr_batch_size", "8",
    '--vr_enable_tta',
    "--output_format", "FLAC",
    "--single_stem", "Vocals",
    '--output_dir', f"{no_back_folder}"
])
filename = get_last_modified_file(no_back_folder)
no_back_output = os.path.join(no_back_folder, filename)
print(f"{filename} processing with karokee_4band_v2_sn is over!")
subprocess.run([
    'audio-separator',
    f"{no_back_output}",
    '--model_file_dir', "/content/arquivos",
    '-m', 'UVR-DeEcho-DeReverb.pth',
    '--vr_window_size', '320',
    '--vr_aggression', '5',
    "--normalization", "1.0",
    "--vr_batch_size", "8",
    '--vr_enable_tta',
    "--output_format", "FLAC",
    "--single_stem", "No Reverb",
    '--output_dir', f"{output_folder}"
])
processed_file = get_last_modified_file(output_folder)
name = os.path.commonprefix(processed_file)
!cp processed_file output_drive
convert_to_mp3(processed_file, name)
print(f"{filename} processing with UVR-DeEcho-DeReverb is over!")
clear_output()
print("Vocal processing completed.")
print("The FLAC file is available in \"RVC AI Cover Maker\" folder on your Drive!")
print("And MP3 in the player below!\n")
display(Audio(name, autoplay = False))